<a href="https://colab.research.google.com/github/joseruga/Estadistica_y_Probabilidad_para-CD-/blob/main/Semana6/Metropolis_Hastings_para_decifrar_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2> Algoritmo Metropolis Hastings para decifrar textos <h2>

El presente código es una adaptación del presentado en [este repositorio](https://github.com/svivek/mcmc-decoding-example)

El ejemplo está inspirado en la historia descrita en [The Markov Chain Monte Carlo Revolution](https://www.ams.org/journals/bull/2009-46-02/S0273-0979-08-01238-X/S0273-0979-08-01238-X.pdf). 







<img src='https://pbs.twimg.com/media/DnJnUCpWwAAmPnR.jpg'>



El objetivo es decifrar los mensajes intercambiados por prisioneros, con la conjetura de que cada simbolo representa una letra o simbolo del lenguaje. La tarea es pues, encontrar ese diccionario o **clave** para codificar y decodificar los mensajes.

## Cifrado por sustitución

Es un método de cifrado para codificarr y decodificar mensajes usando una clave que relaciona una letra o simbolo por otro.

Como ejemplo, se puede suponer que la clave de un cifrado está contenida en la siguiente tabla:

|original | codificado |
|---------|------------|
|a        | z          |
|n        | 4          |
|l        | 5          |
|i        | a          |
|s        | 8          |

De manera que la palabra  `analisis` puede ser codificada como `z4z5a8a8`. 

In [49]:
def code(text, key): 
    return ''.join(list(map(lambda char: key[char], text)))

In [2]:
clave = {'a': 'z', 'n': '4', 'l': '5', 'i': 'a', 's':'8'}
code('analisis', clave)

'z4z5a8a8'

Y podemos decodificar la palabra usando la clave inversa:

In [3]:
inv_clave = {v : k for k, v in clave.items()}
code('z4z5a8a8', inv_clave)

'analisis'

#¿Cómo encontrar una clave adecuada?

Si consideramos solamente el espacio en blanco, las letras en minúscula, y los números estamos hablando de 38 simbolos, y por tanto el espacio muestral en el que vive la clave es enorme.

In [4]:
alphabet = "abcdefghijklmnñopqrstuvwxyz0123456789 "

print('Total de simbolos considerados:', len(alphabet))

from math import factorial
print('Tamaño del espacio muestral en el que vive la clave que buscamos', factorial(38))

Total de simbolos considerados: 38
Tamaño del espacio muestral en el que vive la clave que buscamos 523022617466601111760007224100074291200000000


Supondremos que el lenguaje de los mensajes cifrados es el español, por tanto utilizaremos como guía las probabilidades de transición de los carácteres en español.

# Calificación de las claves candidatas

Para cada secuencia de caracteres $c_1c_2c_3\cdots c_n$, estimaresmos su probabilidad de la siguiente manera

$$P(c_1c_2c_3\cdots c_n) = P(c_1) P(c_2 \mid c_1) P(c_3 \mid c_2) \cdots P(c_n \mid c_{n-1})$$

Así, la probabilidad de la palabra `analisis` se calcularía como 

 $P(a)~P(n \mid a)P(a \mid n)P(l \mid a)P(i \mid l)P(s \mid i)P(i \mid s)P(s \mid i)$.

Para hacer lo anterior necesitamos una matriz de transición de una caracter a otro, y en este sentido, para facilitar el análisis,  ignoraremos el primer caracter porque no hará practicamente diferencia en una secuencia larga.

## ¿Cómo obtener la matriz de transición de los caracteres? 


In [5]:
# ¿Cuántas probabilidades de transición debe tener nuestra matriz de trancisión?
len(alphabet)**2

1444

Para aprender estas probabilidades usaremos un texto, lo más largo posible en español

In [50]:
import re

def limpiar_texto(texto):

    #cambiar texto a minusculas
    texto = texto.lower()
    
    # quitar tildes
    a,b = 'áéíóúü','aeiouu'
    trans = str.maketrans( a,b)
    texto = texto.translate(trans)

    # solo letras y numeros
    texto = re.sub('[^ña-z0-9 ]+' ,' ', texto).strip()  

    # remplazar multiples espacios contiguos por un solo espacio
    texto = re.sub('\s+',' ', texto)                    
    return texto

def find_between( s, first, last ):
    '''fuente: https://stackoverflow.com/questions/3368969/find-string-between-two-substrings'''
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""
def leer_texto(file, inicio=None, fin=None):
    with open(file, 'r') as f:
        if inicio is not None and fin is not None:
            libro = find_between(f.read(), inicio, fin)
        else:
            libro = f.read()
    return limpiar_texto(libro)

Se puede calcular las probabilidades de transisión utilizando cualquier texto. Si este es suficientemente grande, podremos obtener buenas estimaciones. Usaremos Don Quijote de la Mancha, que tiene más de 2 millones de caracteres.

In [51]:
from collections import Counter
import math

def bigram_log_probabilities(book):
    
    character_counts = Counter(book)
    bigram_counts = Counter(zip(book, book[1:]))
    bigrams = bigram_counts.keys()
    
    return dict(map(lambda k: [k, math.log(bigram_counts[k]) - math.log(character_counts[k[0]])], bigrams))

In [8]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [52]:
inicio = "*** START OF THE PROJECT GUTENBERG EBOOK DON QUIJOTE ***"
fin = "*** END OF THE PROJECT GUTENBERG EBOOK DON QUIJOTE ***"

file = '/content/DonQuijote.txt'
DonQ = leer_texto(file, inicio, fin )

In [53]:
bigram_table = bigram_log_probabilities(DonQ)
bigram_table

{('e', 'l'): -2.495635370118805,
 ('l', ' '): -1.633573260510028,
 (' ', 'i'): -4.937384512589171,
 ('i', 'n'): -2.3504318062150116,
 ('n', 'g'): -4.031173508631302,
 ('g', 'e'): -2.980174879019078,
 ('e', 'n'): -1.8827851415801167,
 ('n', 'i'): -3.196628842521836,
 ('i', 'o'): -2.349621480303913,
 ('o', 's'): -1.8348327937786753,
 ('s', 'o'): -2.8913442783423715,
 ('o', ' '): -0.8242763995237041,
 (' ', 'h'): -3.420071451532097,
 ('h', 'i'): -2.3957986266629003,
 ('i', 'd'): -2.801143855311272,
 ('d', 'a'): -2.023135023232147,
 ('a', 'l'): -2.6003410610565005,
 ('l', 'g'): -4.105093437001927,
 ('g', 'o'): -1.4025662217672572,
 (' ', 'd'): -2.166409555430894,
 ('d', 'o'): -1.3743360383401892,
 ('o', 'n'): -2.0723476075124374,
 ('n', ' '): -1.137340896369082,
 (' ', 'q'): -2.633056752874694,
 ('q', 'u'): 0.0,
 ('u', 'i'): -2.480561604574996,
 ('i', 'j'): -2.933481589751146,
 ('j', 'o'): -0.5809899206883387,
 ('o', 't'): -3.485054422488467,
 ('t', 'e'): -1.3776567950215597,
 ('e', ' '): 

Y mediante la siguiente función calcularemos que tan "buena" es la secuencia utilizando para ello las probabilidades de transición estimadas.

In [54]:
LOG_EPSILON = -30  # probabilidad muy pequeña vista como logaritmo, que se ustilizara cuando no exista probabilidad de trancisión de una letra a otra

def goodness(text, bigram_table):
    bigrams = zip(text, text[1:])
    sum = 0
    for b in bigrams:
        sum += bigram_table.get(b) or LOG_EPSILON
    return sum

In [12]:
print("Log probability of 'opamc' = " +str(goodness("opamc", bigram_table)))

Log probability of 'opamc' = -40.784560457172915


In [13]:
print("Log probabilidad de 'campo' = " +str(goodness("campo", bigram_table)))

Log probabilidad de 'campo' = -9.40201613643682


# Implementación

## Creación del mensaje encriptado

In [55]:
plain_text = limpiar_texto(
    """Muchos años después, frente al pelotón de fusilamiento, el coronel Aureliano Buendía había de 
recordar aquella tarde remota en que su padre lo llevó a conocer el hielo. Macondo era entonces 
una aldea de veinte casas de barro y cañabrava construidas a la orilla de un río de aguas diáfanas 
que se precipitaban por un lecho de piedras pulidas, blancas y enormes como huevos 
prehistóricos. El mundo era tan reciente, que muchas cosas carecían de nombre, y para 
mencionarlas había que señalarías con el dedo. Todos los años, por el mes de marzo, una familia 
de gitanos desarrapados plantaba su carpa cerca de la aldea, y con un grande alboroto de pitos y 
timbales daban a conocer los nuevos inventos. Primero llevaron el imán. Un gitano corpulento, de 
barba montaraz y manos de gorrión, que se presentó con el nombre de Melquíades, hizo una 
truculenta demostración pública de lo que él mismo llamaba la octava maravilla de los sabios 
alquimistas de Macedonia. Fue de casa en casa arrastrando dos lingotes metálicos, y todo el 
mundo se espantó al ver que los calderos, las pailas, las tenazas y los anafes se caían de su sitio, 
y las maderas crujían por la desesperación de los clavos y los tornillos tratando de desenclavarse, 
y aun los objetos perdidos desde hacía mucho tiempo aparecían por donde más se les había 
buscado, y se arrastraban en desbandada turbulenta detrás de los fierros mágicos de Melquíades. 
"""
)

In [56]:
import random

def make_random_key():
    l = list(alphabet)
    random.shuffle(l)
    return dict(zip(alphabet, l))

In [37]:
key = make_random_key()
cipher_text = code(plain_text, key)
cipher_text

'xmtprn1a3rn1h4ngm4n1vj45k41a91g49rkr51h41vmn09ax045kr1491trjr5491amj490a5r12m45h0a1pa20a1h41j4trjhaj1a m499a1kajh41j4xrka1451 m41nm1gahj419r1994fr1a1tr5rt4j1491p049r1xatr5hr14ja145kr5t4n1m5a1a9h4a1h41f405k41tanan1h412ajjr1ñ1ta3a2jafa1tr5nkjm0han1a19a1rj099a1h41m51j0r1h41asman1h0ava5an1 m41n41gj4t0g0ka2a51grj1m5194tpr1h41g04hjan1gm90han129a5tan1ñ145rjx4n1trxr1pm4frn1gj4p0nkrj0trn1491xm5hr14ja1ka51j4t045k41 m41xmtpan1trnan1taj4t0a51h415rx2j41ñ1gaja1x45t0r5aj9an1pa20a1 m41n43a9aj0an1tr51491h4hr1krhrn19rn1a3rn1grj1491x4n1h41xajyr1m5a1vax090a1h41s0ka5rn1h4najjagahrn1g9a5ka2a1nm1tajga1t4jta1h419a1a9h4a1ñ1tr51m51sja5h41a92rjrkr1h41g0krn1ñ1k0x2a94n1ha2a51a1tr5rt4j19rn15m4frn105f45krn1gj0x4jr1994fajr514910xa51m51s0ka5r1trjgm945kr1h412aj2a1xr5kajay1ñ1xa5rn1h41srjj0r51 m41n41gj4n45kr1tr514915rx2j41h41x49 m0ah4n1p0yr1m5a1kjmtm945ka1h4xrnkjat0r51gm290ta1h419r1 m41491x0nxr199axa2a19a1rtkafa1xajaf099a1h419rn1na20rn1a9 m0x0nkan1h41xat4hr50a1vm41h41tana1451tana1ajjankja5hr1hrn1905srk4n1x4ka90trn1ñ1krh

## Busqueda de la clave

Empezaremos la busqueda de la clave adecuado, utilizando en el inicio una **clave** aleatoria que iremos cambiando por una  clave cercana, simplemente transponiendo dos caracteres seleccionados aleatoriamente.

In [18]:
clave0 = make_random_key()  # creamos una clave inicial
clave0

{'a': 'm',
 'b': '6',
 'c': 'c',
 'd': 'l',
 'e': '0',
 'f': 'p',
 'g': 't',
 'h': 'n',
 'i': 'b',
 'j': 'ñ',
 'k': 'q',
 'l': 'a',
 'm': '1',
 'n': '4',
 'ñ': '3',
 'o': 'i',
 'p': 'v',
 'q': 'u',
 'r': 'h',
 's': 'w',
 't': 'j',
 'u': 'o',
 'v': '9',
 'w': 'k',
 'x': '8',
 'y': ' ',
 'z': '2',
 '0': 's',
 '1': 'd',
 '2': 'g',
 '3': 'x',
 '4': 'z',
 '5': 'r',
 '6': '7',
 '7': '5',
 '8': 'e',
 '9': 'y',
 ' ': 'f'}

In [19]:
decoded0 =code(cipher_text, clave0) # decodificamos
decoded0

'k4m 5rdgs5rdq7rl47rdhp7fv7dgbdl7b5v5fdq7dh4r3bgk37fv5d7bdm5p5f7bdg4p7b3gf5du47fq3gd gu3gdq7dp7m5pqgpdgj47bbgdvgpq7dp7k5vgd7fdj47dr4dlgqp7db5dbb7i5dgdm5f5m7pd7bd 37b5dkgm5fq5d7pgd7fv5fm7rd4fgdgbq7gdq7di73fv7dmgrgrdq7dugpp5dedmgsgupgigdm5frvp43qgrdgdbgd5p3bbgdq7d4fdp35dq7dgy4grdq3ghgfgrdj47dr7dlp7m3l3vgugfdl5pd4fdb7m 5dq7dl37qpgrdl4b3qgrdubgfmgrded7f5pk7rdm5k5d 47i5rdlp7 3rv5p3m5rd7bdk4fq5d7pgdvgfdp7m37fv7dj47dk4m grdm5rgrdmgp7m3gfdq7df5kup7dedlgpgdk7fm35fgpbgrd gu3gdj47dr7sgbgp3grdm5fd7bdq7q5dv5q5rdb5rdgs5rdl5pd7bdk7rdq7dkgp95d4fgdhgk3b3gdq7dy3vgf5rdq7rgppglgq5rdlbgfvgugdr4dmgplgdm7pmgdq7dbgdgbq7gdedm5fd4fdypgfq7dgbu5p5v5dq7dl3v5rdedv3kugb7rdqgugfdgdm5f5m7pdb5rdf47i5rd3fi7fv5rdlp3k7p5dbb7igp5fd7bd3kgfd4fdy3vgf5dm5pl4b7fv5dq7dugpugdk5fvgpg9dedkgf5rdq7dy5pp35fdj47dr7dlp7r7fv5dm5fd7bdf5kup7dq7dk7bj43gq7rd 395d4fgdvp4m4b7fvgdq7k5rvpgm35fdl4ub3mgdq7db5dj47d7bdk3rk5dbbgkgugdbgd5mvgigdkgpgi3bbgdq7db5rdrgu35rdgbj43k3rvgrdq7dkgm7q5f3gdh47dq7dmgrgd7fdmgrgdgppgrvpgfq5dq5rdb3fy5v7rdk7vgb3m5rdedv5q

In [20]:
log_p0 = goodness(decoded0, bigram_table)  # evaluamos
log_p0

-32984.91082028115

In [21]:
pair = random.sample(alphabet, 2)
pair[0], pair[1]

('m', '9')

In [22]:
clave0[pair[0]] ,clave0[pair[1]]

('1', 'y')

In [23]:
clave1 = clave0.copy()
clave1[pair[0]] = clave0[pair[1]]
clave1[pair[1]] = clave0[pair[0]]
clave1

{'a': 'm',
 'b': '6',
 'c': 'c',
 'd': 'l',
 'e': '0',
 'f': 'p',
 'g': 't',
 'h': 'n',
 'i': 'b',
 'j': 'ñ',
 'k': 'q',
 'l': 'a',
 'm': 'y',
 'n': '4',
 'ñ': '3',
 'o': 'i',
 'p': 'v',
 'q': 'u',
 'r': 'h',
 's': 'w',
 't': 'j',
 'u': 'o',
 'v': '9',
 'w': 'k',
 'x': '8',
 'y': ' ',
 'z': '2',
 '0': 's',
 '1': 'd',
 '2': 'g',
 '3': 'x',
 '4': 'z',
 '5': 'r',
 '6': '7',
 '7': '5',
 '8': 'e',
 '9': '1',
 ' ': 'f'}

In [24]:
decoded1 =code(cipher_text, clave1) # decodificamos
decoded1

'k4m 5rdgs5rdq7rl47rdhp7fv7dgbdl7b5v5fdq7dh4r3bgk37fv5d7bdm5p5f7bdg4p7b3gf5du47fq3gd gu3gdq7dp7m5pqgpdgj47bbgdvgpq7dp7k5vgd7fdj47dr4dlgqp7db5dbb7i5dgdm5f5m7pd7bd 37b5dkgm5fq5d7pgd7fv5fm7rd4fgdgbq7gdq7di73fv7dmgrgrdq7dugpp5dedmgsgupgigdm5frvp43qgrdgdbgd5p3bbgdq7d4fdp35dq7dg14grdq3ghgfgrdj47dr7dlp7m3l3vgugfdl5pd4fdb7m 5dq7dl37qpgrdl4b3qgrdubgfmgrded7f5pk7rdm5k5d 47i5rdlp7 3rv5p3m5rd7bdk4fq5d7pgdvgfdp7m37fv7dj47dk4m grdm5rgrdmgp7m3gfdq7df5kup7dedlgpgdk7fm35fgpbgrd gu3gdj47dr7sgbgp3grdm5fd7bdq7q5dv5q5rdb5rdgs5rdl5pd7bdk7rdq7dkgp95d4fgdhgk3b3gdq7d13vgf5rdq7rgppglgq5rdlbgfvgugdr4dmgplgdm7pmgdq7dbgdgbq7gdedm5fd4fd1pgfq7dgbu5p5v5dq7dl3v5rdedv3kugb7rdqgugfdgdm5f5m7pdb5rdf47i5rd3fi7fv5rdlp3k7p5dbb7igp5fd7bd3kgfd4fd13vgf5dm5pl4b7fv5dq7dugpugdk5fvgpg9dedkgf5rdq7d15pp35fdj47dr7dlp7r7fv5dm5fd7bdf5kup7dq7dk7bj43gq7rd 395d4fgdvp4m4b7fvgdq7k5rvpgm35fdl4ub3mgdq7db5dj47d7bdk3rk5dbbgkgugdbgd5mvgigdkgpgi3bbgdq7db5rdrgu35rdgbj43k3rvgrdq7dkgm7q5f3gdh47dq7dmgrgd7fdmgrgdgppgrvpgfq5dq5rdb3f15v7rdk7vgb3m5rdedv5q

In [25]:
log_p1 = goodness(decoded1, bigram_table)  # evaluamos
log_p1

-32901.14914490618

En base a lo anterior ¿Aceptamos la nueva clave propuesta?

## Algoritmo completo

In [57]:
def transpose_random(key):
    pair = random.sample(alphabet, 2)
    new_key = dict(key)
    new_key[pair[0]] = key[pair[1]]
    new_key[pair[1]] = key[pair[0]]
    return new_key

def decode(cipher_text, bigram_table, iters = 100000, print_every = 10000):
    # 1. inicializamos con una clave aleatoria
    current_key = make_random_key()  

    for i in range(0, iters):  # en cada iteración utilizaremos una nueva clave

        # 2. decodificamos la secuencia con la clave
        decoded = code(cipher_text, current_key) 

        if i % print_every == 0:
            print(str(i) + "\t" + decoded + "\n")

        # 3. evaluamos la secuencia utilizando el modelo de bigramas 
        score = goodness(decoded, bigram_table)   
               
        # 4. proponemos una nueva clave "cercana"
        changed_key = transpose_random(current_key) 

        # 5. evaluamos la nueva clave decodificando la secuencia  
        changed_score = goodness(code(cipher_text, changed_key), bigram_table)  # evaluamos la secuencia con la nueva clave propuesta

        # 6.1 si la clave propuesta da mejores resultados nos quedamos con ella para la prox iteración
        if changed_score > score:   
            current_key = changed_key

        # damos oportunidad proporcional a la relación entre la probabilidad de las dos secuencias evaluadas 
        else:  
            diff = changed_score - score
            if math.log(random.random())  < diff:
                current_key = changed_key

    decoded = code(cipher_text, current_key)
    print("Final decoded: " + decoded)
    return decoded

In [58]:
decode(cipher_text, bigram_table)

0	was53b 3uañ32j2bñj03sa2b jñjbqñaañb9iaba8b 8jsawjbua8bqij8b1asnjba8b ñnsqn nw3bañjba8bj2wañ3nuabdbp6lba2wabj2wañ3nuab7jb2nu3b1n2w3b2383bisjb1a0bq3sba8bwa8a2q3 n3basb6fofb 3ñbisbj2wñ3s3k3bwiñq3ba2wabj2wñ3s3k3b7n03bisjb5ñjsbuak32wñjqn3sbuab2ibua2qidñnknasw3basbisbq3s5ña23bnswañsjqn3sj8buabj2wñ3s3knjb añ3bsjunab8abqñay3bjbqji2jbuab2ibkjsañjbuab1a2wnñb8j2b añ23sj2bkjy3ña2b23sbj2nbta8n0kaswab jñjb8jbña iwjqn3sbua8bj2wañ3nuabdbp6lbisbunqwju3ñbwiñq3bnk i23bjb2ib iad83bdj43b asjbuabkiañwaba8b1a2wnu3bjb8jbaiñ3 ajbasw3sqa2ba8bj2wñ3s3k3b1381n3bjbujñbqiaswjbuab2ibua2qidñnknasw3basb6flobybq3k3b8iqnjbisbwñj4abkiyba8a5jswabw3u3ba8bkisu3bjqa w3b2ibuak32wñjqn3s

10000	rengo potelobab lazoneb pala cleel fue es psanera tes cuas venia es plincipiro ela es abreloite d q10 ebre abreloite ha bito vibro boso una vez con es resebcopio en 1646 pol un abrlonomo rulco ebre abrlonomo hizo una glan temobrlacion te bu tebcudlimienro en un conglebo inrelnacionas te abrlonomia pelo natie se cleyo a cauba te bu manel

'rengo potelobab lazoneb pala cleel jue es psanera tes cuas venia es plincipiro ela es abreloite d q10 ebre abreloite ha bito vibro boso una vez con es resebcopio en 1646 pol un abrlonomo rulco ebre abrlonomo hizo una glan temobrlacion te bu tebcudlimienro en un conglebo inrelnacionas te abrlonomia pelo natie se cleyo a cauba te bu manela te vebril sab pelbonab mayoleb bon abi fesizmenre pala sa lepuracion tes abreloite d q10 un ticratol rulco impubo a bu puedso daño pena te muelre es vebrito a sa eulopea enronceb es abrlonomo vosvio a tal cuenra te bu tebcudlimienro en 1604 y como sucia un rlañe muy eseganre roto es munto acepro bu temobrlacion'

# Otros ejemplos

In [59]:
# El principito. Tomado de https://archive.org/stream/ElPrincipitoAntoineDeSaintExupery/El_principito-_Antoine_De_Saint_Exupery_djvu.txt

plain_text = limpiar_texto(
    """Tengo poderosas razones para creer que el planeta del cual venía el principito era 
el asteroide B 612. Este asteroide ha sido visto sólo una vez con el telescopio en 
1909, por un astrónomo turco. 
Este astrónomo hizo una gran demostración de su descubrimiento en un congreso 
Internacional de Astronomía. Pero nadie le creyó a causa de su manera de vestir. 
Las personas mayores son así. Felizmente para la reputación del asteroide B 612, 
un dictador turco impuso a su pueblo, bajo pena de muerte, el vestido a la 
europea. Entonces el astrónomo volvió a dar cuenta de su descubrimiento en 1920 
y como lucía un traje muy elegante, todo el mundo aceptó su demostración. 
"""
)
cipher_text = code(plain_text, key)
cipher_text

'k45sr1grh4jrnan1jayr54n1gaja1tj44j1 m41491g9a54ka1h491tma91f450a1491gj05t0g0kr14ja1491ank4jr0h4121cb814nk41ank4jr0h41pa1n0hr1f0nkr1nr9r1m5a1f4y1tr51491k494ntrg0r1451b6w61grj1m51ankjr5rxr1kmjtr14nk41ankjr5rxr1p0yr1m5a1sja51h4xrnkjat0r51h41nm1h4ntm2j0x045kr1451m51tr5sj4nr105k4j5at0r5a91h41ankjr5rx0a1g4jr15ah041941tj4ñr1a1tamna1h41nm1xa54ja1h41f4nk0j19an1g4jnr5an1xañrj4n1nr51an01v490yx45k41gaja19a1j4gmkat0r51h491ank4jr0h4121cb81m51h0tkahrj1kmjtr10xgmnr1a1nm1gm429r12aur1g45a1h41xm4jk41491f4nk0hr1a19a14mjrg4a145kr5t4n1491ankjr5rxr1fr9f0r1a1haj1tm45ka1h41nm1h4ntm2j0x045kr1451b68w1ñ1trxr19mt0a1m51kjau41xmñ1494sa5k41krhr1491xm5hr1at4gkr1nm1h4xrnkjat0r5'

In [60]:
decode(cipher_text, bigram_table)

0	s2y5d10dt2cdege1cgpdy2e10gcg14c22c1iv212o10ogy2sg1t2o14vgo1h2y8g12o10c8y4808sd12cg12o1ges2cd8t21917añ12es21ges2cd8t21xg1e8td1h8esd1edod1vyg1h2p14dy12o1s2o2e4d08d12y1arzr10dc1vy1gescdydld1svc4d12es21gescdydld1x8pd1vyg15cgy1t2ldescg48dy1t21ev1t2e4v9c8l82ysd12y1vy14dy5c2ed18ys2cyg48dygo1t21gescdydl8g102cd1ygt821o214c26d1g14gveg1t21ev1lgy2cg1t21h2es8c1oge102cedyge1lg6dc2e1edy1ge81m2o8pl2ys210gcg1og1c20vsg48dy1t2o1ges2cd8t21917añ1vy1t84sgtdc1svc4d18l0ved1g1ev10v29od19gkd102yg1t21lv2cs212o1h2es8td1g1og12vcd02g12ysdy42e12o1gescdydld1hdoh8d1g1tgc14v2ysg1t21ev1t2e4v9c8l82ysd12y1arñz1614dld1ov48g1vy1scgk21lv612o25gys21sdtd12o1lvytd1g420sd1ev1t2ldescg48dy

10000	ienfo cotelosas lazones cala gleel mue ed cdaneia ted guad henra ed clrngrcrio ela ed asielorte x q10 esie asielorte ba srto hrsio sodo una hez gon ed iedesgocro en 1646 col un asilonopo iulgo esie asilonopo brzo una flan teposilagron te su tesguxlrprenio en un gonfleso rnielnagronad te asilonopra celo natre de gleyo a gausa te su panel

'ianfo cotaloses lemonas cele glaal vua ad cdenaie tad gued hanre ad clrngrcrio ale ad esialorta y q10 asia esialorta be srto hrsio sodo une ham gon ad iadasgocro an 16x6 col un esilonopo iulgo asia esilonopo brmo une flen taposilegron ta su tasguylrpranio an un gonflaso rnialnegroned ta esilonopre calo netra da glazo e geuse ta su penale ta hasirl des calsones pezolas son esr jadrmpania cele de lacuiegron tad esialorta y q10 un trgietol iulgo rpcuso e su cuaydo yeño cane ta pualia ad hasirto e de aulocae aniongas ad esilonopo hodhro e tel guanie ta su tasguylrpranio an 160x z gopo dugre un ileña puz adafenia ioto ad punto egacio su taposilegron'

In [61]:
# La liebre y la tortuga. Tomado de https://psicologiaymente.com/cultura/fabulas-de-esopo

plain_text = limpiar_texto(
    """Un día una liebre orgullosa y veloz, vió como una tortuga caminaba por el camino y se le acercó. La liebre empezó a burlarse de la lentitud del otro animal y de la longitud de sus patas. Sin embargo, la tortuga le respondió que estaba segura de que a pesar de la gran velocidad de la liebre era capaz de ganarla en una carrera.
La liebre, segura de su victoria y considerando el reto imposible de perder, aceptó. Ambos pidieron a la zorra que señalara la meta, a lo que esta aceptó, al igual que al cuervo para que hiciera de juez.
Al llegar el día de la competición, al empezar la carrera la liebre y la tortuga salieron al mismo tiempo. La tortuga avanzaba sin detenerse, pero lentamente.
La liebre era muy veloz, y viendo que sacaba una gran ventaja a la tortuga decidió ir parándose y descansando de vez en cuando. Pero en una de las ocasiones la liebre se quedó dormida. La tortuga, poco a poco, siguió avanzando.
Cuando la liebre despertó, se encontró con que la tortuga estaba a punto de cruzar la meta. Aunque echó a correr fue demasiado tarde y finalmente la tortuga ganó la carrera".
Esta fábula nos enseña que el trabajo duro, la perseverancia, la constancia y el esfuerzo nos llevarán a nuestras metas, aunque sea poco a poco, si no nos rendimos. También nos permite ver cómo la arrogancia, la falta de constancia y el exceso de seguridad en uno mismo nos puede llevar a perder oportunidades y a no alcanzar nuestras metas.
"""
)
cipher_text = code(plain_text, key)
cipher_text

'm51h0a1m5a19042j41rjsm99rna1ñ1f49ry1f0r1trxr1m5a1krjkmsa1tax05a2a1grj1491tax05r1ñ1n41941at4jtr19a19042j414xg4yr1a12mj9ajn41h419a1945k0kmh1h491rkjr1a50xa91ñ1h419a19r5s0kmh1h41nmn1gakan1n0514x2ajsr19a1krjkmsa1941j4ngr5h0r1 m414nka2a1n4smja1h41 m41a1g4naj1h419a1sja51f49rt0hah1h419a19042j414ja1tagay1h41sa5aj9a1451m5a1tajj4ja19a19042j41n4smja1h41nm1f0tkrj0a1ñ1tr5n0h4ja5hr1491j4kr10xgrn02941h41g4jh4j1at4gkr1ax2rn1g0h04jr51a19a1yrjja1 m41n43a9aja19a1x4ka1a19r1 m414nka1at4gkr1a910sma91 m41a91tm4jfr1gaja1 m41p0t04ja1h41um4y1a91994saj1491h0a1h419a1trxg4k0t0r51a914xg4yaj19a1tajj4ja19a19042j41ñ19a1krjkmsa1na904jr51a91x0nxr1k04xgr19a1krjkmsa1afa5ya2a1n051h4k454jn41g4jr1945kax45k419a19042j414ja1xmñ1f49ry1ñ1f045hr1 m41nata2a1m5a1sja51f45kaua1a19a1krjkmsa1h4t0h0r10j1gaja5hrn41ñ1h4nta5na5hr1h41f4y1451tma5hr1g4jr1451m5a1h419an1rtan0r54n19a19042j41n41 m4hr1hrjx0ha19a1krjkmsa1grtr1a1grtr1n0sm0r1afa5ya5hr1tma5hr19a19042j41h4ng4jkr1n4145tr5kjr1tr51 m419a1krjkmsa14nka2a1a1gm5kr1h41tjmyaj19a1x4ka1am5 m414tpr

In [62]:
decode(cipher_text, bigram_table)

0	97ub2 u97 uo2l5yluayk9ooax utueloasue2aufarau97 uvayv9k uf r27 5 u3ayulouf r27autuxluolu flyfauo uo2l5ylulr3lsau u59yo yxlubluo uol7v2v9bublouavyau 72r outubluo uoa7k2v9bublux9xu3 v xux27ulr5 ykauo uvayv9k uoluylx3a7b2auw9lulxv 5 uxlk9y ubluw9lu u3lx yubluo uky 7ueloaf2b bubluo uo2l5yluly uf 3 subluk 7 yo ul7u97 uf yyly uo uo2l5yluxlk9y ublux9ue2fvay2 utufa7x2bly 7baulouylvau2r3ax25olublu3lyblyu fl3vau r5axu32b2lya7u uo usayy uw9luxlm o y uo urlv u uoauw9lulxv u fl3vau ou2k9 ouw9lu ouf9lyeau3 y uw9lu02f2ly ubluz9lsu ouoolk yuloub2 ubluo ufar3lv2f2a7u oulr3ls yuo uf yyly uo uo2l5ylutuo uvayv9k ux o2lya7u our2xrauv2lr3auo uvayv9k u e 7s 5 ux27ublvl7lyxlu3lyauol7v rl7vluo uo2l5yluly ur9tueloasutue2l7bauw9lux f 5 u97 uky 7uel7v z u uo uvayv9k ublf2b2au2yu3 y 7baxlutublxf 7x 7baubluelsul7uf9 7bau3lyaul7u97 ubluo xuaf x2a7lxuo uo2l5yluxluw9lbaubayr2b uo uvayv9k u3afau u3afaux2k92au e 7s 7bauf9 7bauo uo2l5ylublx3lyvauxlul7fa7vyaufa7uw9luo uvayv9k ulxv 5 u u397vaublufy9s yuo urlv u 97w9lulf0

'un dia una liebre orgullosa y veloz vio como una tortuga caminaba por el camino y se le acerco la liebre empezo a burlarse de la lentitud del otro animal y de la longitud de sus patas sin embargo la tortuga le respondio fue estaba segura de fue a pesar de la gran velocidad de la liebre era capaz de ganarla en una carrera la liebre segura de su victoria y considerando el reto imposible de perder acepto ambos pidieron a la zorra fue señalara la meta a lo fue esta acepto al igual fue al cuervo para fue xiciera de juez al llegar el dia de la competicion al empezar la carrera la liebre y la tortuga salieron al mismo tiempo la tortuga avanzaba sin detenerse pero lentamente la liebre era muy veloz y viendo fue sacaba una gran ventaja a la tortuga decidio ir parandose y descansando de vez en cuando pero en una de las ocasiones la liebre se fuedo dormida la tortuga poco a poco siguio avanzando cuando la liebre desperto se encontro con fue la tortuga estaba a punto de cruzar la meta aunfue ecxo